## MLflow's Model Registry

In [3]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [85]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
# client.list_experiments() Deprecated apparently
client.search_experiments()

[<Experiment: artifact_location='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/2', creation_time=1685120904748, experiment_id='2', last_update_time=1685120904748, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1', creation_time=1684888611882, experiment_id='1', last_update_time=1684888611882, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/0', creation_time=1684888611850, experiment_id='0', last_update_time=1684888611850, lifecycle_stage='active', name='Default', tags={}>]

In [12]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [57]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=8,
    order_by=["metrics.rmse ASC"]
)

In [58]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")
    # print(f"params: {run.data.params}")
    # print(f"tags: {run.data.tags}")
    # print(f"status: {run.info}")                                                                                                                                                                                   

run id: d0b1526fbb3c4161b2ae442c0f246bfe, rmse: 6.2830
run id: 4202850f89504c53a4ac466f1c14499a, rmse: 6.2830
run id: c95a9f747fa54c7ca96c182b7674477a, rmse: 6.2830
run id: 71fdd7d5270d4278b938a820502fca21, rmse: 6.2830
run id: 0427cea87fd94154827eba1d7e0c9409, rmse: 6.2830
run id: 2e076262f75842fa83c94b638e6cc1e3, rmse: 6.2830
run id: ce4e759cb182448688f2eea228892ba5, rmse: 6.2869
run id: 1d504ddfc9844565a27e83cd6f868704, rmse: 6.2881


In [62]:
import os
for run in runs:
    model_file_path = run.info.artifact_uri + "/models_mlflow/model.xgb"
    if not os.path.isfile(model_file_path):
        print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}, model file size: model not saved")
    else:
        print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}, model file size: {os.path.getsize(model_file_path) / 1024 / 1024:.1f} MB")

run id: d0b1526fbb3c4161b2ae442c0f246bfe, rmse: 6.2830, model file size: 4.3 MB
run id: 4202850f89504c53a4ac466f1c14499a, rmse: 6.2830, model file size: 4.3 MB
run id: c95a9f747fa54c7ca96c182b7674477a, rmse: 6.2830, model file size: 4.3 MB
run id: 71fdd7d5270d4278b938a820502fca21, rmse: 6.2830, model file size: 4.3 MB
run id: 0427cea87fd94154827eba1d7e0c9409, rmse: 6.2830, model file size: 4.3 MB
run id: 2e076262f75842fa83c94b638e6cc1e3, rmse: 6.2830, model file size: model not saved
run id: ce4e759cb182448688f2eea228892ba5, rmse: 6.2869, model file size: model not saved
run id: 1d504ddfc9844565a27e83cd6f868704, rmse: 6.2881, model file size: model not saved


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [113]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [43]:
run_id = "d0b1526fbb3c4161b2ae442c0f246bfe"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/26 14:02:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685124154051, current_stage='None', description=None, last_updated_timestamp=1685124154051, name='nyc-taxi-regressor', run_id='d0b1526fbb3c4161b2ae442c0f246bfe', run_link=None, source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/d0b1526fbb3c4161b2ae442c0f246bfe/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [66]:
client.get_registered_model(name="nyc-taxi-regressor")

<RegisteredModel: aliases={}, creation_timestamp=1685050712544, description='The NYC Taxi Regressor for Ride Duration', last_updated_timestamp=1685124154051, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685050712648, current_stage='Staging', description='', last_updated_timestamp=1685050991934, name='nyc-taxi-regressor', run_id='d0b1526fbb3c4161b2ae442c0f246bfe', run_link='', source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/d0b1526fbb3c4161b2ae442c0f246bfe/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1685124154051, current_stage='None', description=None, last_updated_timestamp=1685124154051, name='nyc-taxi-regressor', run_id='d0b1526fbb3c4161b2ae442c0f246bfe', run_link=None, source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/d0b1526fbb3c4161b2

In [67]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


In [68]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685124154051, current_stage='Staging', description=None, last_updated_timestamp=1685150274621, name='nyc-taxi-regressor', run_id='d0b1526fbb3c4161b2ae442c0f246bfe', run_link=None, source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/d0b1526fbb3c4161b2ae442c0f246bfe/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [69]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging


In [70]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685124154051, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2023-05-26', last_updated_timestamp=1685150402461, name='nyc-taxi-regressor', run_id='d0b1526fbb3c4161b2ae442c0f246bfe', run_link=None, source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/d0b1526fbb3c4161b2ae442c0f246bfe/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [99]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [100]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [114]:
client.get_registered_model(name=model_name)

<RegisteredModel: aliases={}, creation_timestamp=1685050712544, description='The NYC Taxi Regressor for Ride Duration', last_updated_timestamp=1685152330259, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685124154051, current_stage='Archived', description='The model version 2 was transitioned to Staging on 2023-05-26', last_updated_timestamp=1685152212108, name='nyc-taxi-regressor', run_id='d0b1526fbb3c4161b2ae442c0f246bfe', run_link=None, source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/d0b1526fbb3c4161b2ae442c0f246bfe/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>,
 <ModelVersion: aliases=[], creation_timestamp=1685151250429, current_stage='Production', description='', last_updated_timestamp=1685151264588, name='nyc-taxi-regressor', run_id='40176c2067c1492ebbb31a639416d119', run_link='', source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experim

In [115]:
run_id = '40176c2067c1492ebbb31a639416d119'

In [116]:
# client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.') # Deprecated
mlflow.artifacts.download_artifacts(run_id=run_id, artifact_path='preprocessor', dst_path='.')

'/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/preprocessor'

In [117]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [118]:
X_test = preprocess(df, dv)

In [119]:
target = "duration"
y_test = df[target].values

In [120]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/26 21:53:13 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 8.84 s, sys: 574 ms, total: 9.41 s
Wall time: 11.5 s


{'rmse': 6.884380652609624}

In [121]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/26 21:53:24 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[21:53:26] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 17.7 s, sys: 302 ms, total: 18 s
Wall time: 4.61 s


{'rmse': 6.233843586198147}

In [122]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685152299364, current_stage='Production', description='', last_updated_timestamp=1685152866587, name='nyc-taxi-regressor', run_id='71fdd7d5270d4278b938a820502fca21', run_link='', source='/mnt/c/Users/mrozt/Documents/DataTalksClub/mlops-zoomcamp-course/02-experiment-tracking/mlruns/1/71fdd7d5270d4278b938a820502fca21/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>